### Configurando caminhos


In [1]:
# URL da fonte de dados
URL = 'https://aplicacoes.mds.gov.br/sagi/servicos/misocial/?fq=anomes_s:2024*&fl=codigo_ibge%2Canomes_s%2Cqtd_familias_beneficiarias_bolsa_familia_s%2Cvalor_repassado_bolsa_familia_s%2Cpbf_vlr_medio_benef_f&fq=valor_repassado_bolsa_familia_s%3A*&q=*%3A*&rows=100000&sort=anomes_s%20desc%2C%20codigo_ibge%20asc&wt=csv'
# Caminho local onde o arquivo será salvo
workdir = './work/'
file = 'bolsa-familia-2024.csv'
# Caminho do driver jdbc
jdbc_driver_path = "/opt/trabalhos/etl-mdd/postgresql-42.7.1.jar"
file_path = workdir + file




### Importando JSON com informações de UF e Município

In [2]:
import json

json_file_path = './utils/uf-code.json'
uf_dict = {}

with open(json_file_path, 'r') as file:
    uf_dict = json.load(file)

print(uf_dict)

{'12': 'Acre', '27': 'Alagoas', '16': 'Amapá', '13': 'Amazonas', '29': 'Bahia', '23': 'Ceará', '53': 'Distrito Federal', '32': 'Espírito Santo', '52': 'Goiás', '21': 'Maranhão', '51': 'Mato Grosso', '50': 'Mato Grosso do Sul', '31': 'Minas Gerais', '15': 'Pará', '25': 'Paraíba', '41': 'Paraná', '26': 'Pernambuco', '22': 'Piauí', '24': 'Rio Grande do Norte', '43': 'Rio Grande do Sul', '33': 'Rio de Janeiro', '11': 'Rondônia', '14': 'Roraima', '42': 'Santa Catarina', '35': 'São Paulo', '28': 'Sergipe', '17': 'Tocantins'}


### Instalando pyspark


In [3]:
%pip install --user pyspark

Note: you may need to restart the kernel to use updated packages.


### Criando sessão spark

In [4]:
from pyspark.sql import SparkSession
# Cria a sessão spark
spark_session = SparkSession.builder.appName('spark') \
                                    .config("spark.driver.extraClassPath", jdbc_driver_path) \
                                    .config("spark.executor.memory", "4g") \
                                    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/17 19:48:28 WARN Utils: Your hostname, RRNWRESID05 resolves to a loopback address: 127.0.1.1; using 172.23.195.127 instead (on interface eth0)
24/06/17 19:48:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 19:48:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Lendo arquivo csv e montando dataframe do spark

In [5]:
df = spark_session.read.options(header="true", delimiter=",", encoding="ISO-8859-1", inferSchema=True).csv(file_path)

In [6]:
# Retirado de: https://medium.com/@salibi/como-validar-o-c%C3%B3digo-de-munic%C3%ADpio-do-ibge-90dc545cc533#:~:text=O%20C%C3%B3digo%20de%20Munic%C3%ADpio%20do%20IBGE%20%C3%A9%20um%20identificador%20%C3%BAnico,o%20%C3%BAltimo%20d%C3%ADgito%2C%20um%20verificador.

def last_digit_ibge(cod6: str):
   a = int(cod6[0])
   b = (int(cod6[1]) * 2) % 10 + (int(cod6[1]) * 2) // 10
   c = int(cod6[2])
   d = (int(cod6[3]) * 2) % 10 + (int(cod6[3]) * 2) // 10
   e = int(cod6[4])
   f = (int(cod6[5]) * 2) % 10 + (int(cod6[5]) * 2) // 10
   digit = (10 - (a + b + c + d + e + f) % 10) % 10
   return str(digit)

# city_code = "355030"
# result = city_code + last_digit_ibge(city_code)
# print(result)


### Imprimindo primeiras linhas do dataframe

In [7]:
# first_column_df = df.select("codigo_ibge")
# first_column_df.show()
df.show()

+-----------+---------+-------------------------------------------+--------------------------------+----------------------+
|codigo_ibge| anomes_s| qtd_familias_beneficiarias_bolsa_familia_s| valor_repassado_bolsa_familia_s| pbf_vlr_medio_benef_f|
+-----------+---------+-------------------------------------------+--------------------------------+----------------------+
|     110001| 202406.0|                                     1705.0|                       1176837.0|                690.63|
|     110002| 202406.0|                                     7494.0|                       5117672.0|                683.82|
|     110003| 202406.0|                                      323.0|                        214990.0|                 665.6|
|     110004| 202406.0|                                     6105.0|                       4127413.0|                676.18|
|     110005| 202406.0|                                     1203.0|                        862839.0|                718.43|
|     11

### Criando coluna para UF

In [8]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

def categorize_age(uf):
    return  uf_dict.get(str(uf)[0:2]) 

categorize_age_udf = udf(categorize_age, StringType())

df = df.withColumn("uf", categorize_age_udf(col("codigo_ibge")))


24/06/17 19:48:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
df.show()

+-----------+---------+-------------------------------------------+--------------------------------+----------------------+--------+
|codigo_ibge| anomes_s| qtd_familias_beneficiarias_bolsa_familia_s| valor_repassado_bolsa_familia_s| pbf_vlr_medio_benef_f|      uf|
+-----------+---------+-------------------------------------------+--------------------------------+----------------------+--------+
|     110001| 202406.0|                                     1705.0|                       1176837.0|                690.63|Rondônia|
|     110002| 202406.0|                                     7494.0|                       5117672.0|                683.82|Rondônia|
|     110003| 202406.0|                                      323.0|                        214990.0|                 665.6|Rondônia|
|     110004| 202406.0|                                     6105.0|                       4127413.0|                676.18|Rondônia|
|     110005| 202406.0|                                     1203.0|  

### Conversão do dataframe para o modelo estrela

In [ ]:
import pyspark.sql.functions as F
from typing import Dict, List
from pyspark.sql.dataframe import DataFrame

def get_columns_list_from_dimension(dimension: Dict[str, List[str]]):
    return [col for cols in dimension for col in cols]

def get_table_name_and_records(dataframe: DataFrame, dimension_table_name_and_columns: Dict[str, List[str]]) -> List[tuple[str, DataFrame]]:
    dimensions = []

    for dimension_table_name, dimension_columns in dimension_table_name_and_columns:
        dimension_records = dataframe.select(*dimension_columns).distinct()
        surrogate_key_column_name = f"sk_{dimension_table_name.replace('dim_', '')}"

        # add unique and increasing id to dimension (but not consecutive)
        unique_and_increasing_id = F.monotonically_increasing_id()
        dimension_records = dimension_records.withColumn(
            surrogate_key_column_name,
            unique_and_increasing_id
        )

        dimension_table_in_tuple = (dimension_table_name, dimension_records)

        dimensions.append(dimension_table_in_tuple)
    
    return dimensions


def transform_spark_dataframe_into_star_schema(
    original_dataframe: DataFrame,
    fact_columns: List[str]  = ["col1", "col2"],
    fact_table_name = "tabela_fato",
    mapping_dimension_columns: Dict[str, List[str]] = {'dim1':["col3", "col4"], "dim2":["col5", "col6"]},
):
    dimension_columns_separated_by_dimension = mapping_dimension_columns.values()

    dimension_columns = get_columns_list_from_dimension(dimension_columns_separated_by_dimension)

    columns_from_fact_and_dimension = fact_columns + dimension_columns

    original_dataframe = original_dataframe.select(*columns_from_fact_and_dimension)

    dimension_table_name_and_columns = mapping_dimension_columns.items()

    dimensions = get_table_name_and_records(original_dataframe, dimension_table_name_and_columns)

    # Substitui as colunas de dimensão pelo respectivo SK na tabela fato
    # ------------------------------------------------------------------
    for table_name, records in dimensions:
        # join the dimension dataframe to the original dataframe
        dimension_columns_by_dimension_from_dataframe = [
            original_dataframe[column] == records[column]
            for column in mapping_dimension_columns[table_name]
        ]
        
        original_dataframe = original_dataframe.join(
            F.broadcast(records), 
            on=dimension_columns_by_dimension_from_dataframe,
            how="left"
        )

    # drop the original columns
    original_dataframe = original_dataframe.drop(*dimension_columns)

    fact_table = (fact_table_name, original_dataframe)
    
    return dimensions + [fact_table]

In [ ]:
star_schema = transform_spark_dataframe_into_star_schema(
    selected_columns_df,
    fact_columns=["QT_VOTOS_NOMINAIS_VALIDOS", "QT_VOTOS_NOMINAIS"],
    fact_table_name="tabela_fato",
    mapping_dimension_columns={
        'dim_municipio': ["SG_UF", "NM_MUNICIPIO"],
        'dim_cargo': ["DS_CARGO"],
        'dim_ds_eleicao':["DS_ELEICAO"],
        'dim_partido':["SG_PARTIDO","NM_PARTIDO", "NR_PARTIDO"],
        'dim_candidato':["NM_CANDIDATO", "NR_CANDIDATO", "NM_URNA_CANDIDATO"],
        'dim_turno':["NR_TURNO"],
        'dim_tp_agrangencia':["TP_ABRANGENCIA"],
        'dim_zona':["NR_ZONA"],
        'dim_situacao_candidatura':["DS_SITUACAO_CANDIDATURA"],
        'dim_coligacao':["NM_COLIGACAO", "DS_COMPOSICAO_COLIGACAO"],
        "dim_voto_transito":["ST_VOTO_EM_TRANSITO"],
        'dim_situacaof_turno':["DS_SIT_TOT_TURNO"],
        'dim_destinacao_voto':["NM_TIPO_DESTINACAO_VOTOS"]
    },   
)

### Configurando conexão com o banco

In [ ]:
hostname_or_ip = "34.172.175.190"
port = "443"
db = "metabase"
user = "star"
password = "star"

db_url = "jdbc:postgresql://" + hostname_or_ip + ":" + port + "/" + db

properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver", 
}


### Transferindo modelo estrela para o banco

In [ ]:
for item in star_schema:
    table_name,dataframe = item
    print(f"Writing {table_name} to Eleicoes DB")
    if table_name == "dim_municipio":
        dataframe.write.jdbc(url=db_url, table=table_name, mode="overwrite", properties=properties)

### Desalocando sessão do spark


In [ ]:
# Stopping spark session
spark_session.stop()

# Cleaning up files 
# Delete the directory and all its contents
# import shutil

# shutil.rmtree(workdir+'extracted/')